# Get config jsons from fine-tuning experiments and analyze

Read the excel files created by the analyze_from_configs. Convert and other things that we can do directly from them


In [1]:
import json, os, sys, io, shutil
from pathlib import Path
import pandas as pd
from datetime import date
from collections import Counter


In [5]:
# Get timestamps and tasks
xl_path = Path("excels")

name_filter = "03121434"
excels = [x for x in xl_path.iterdir() if x.name.startswith(name_filter)]

print([x.name for x in excels])



['03121434_full-report.xlsx', '03121434_best-report.xlsx', '03121434_search-space.xlsx']


In [46]:
search_space = [p for p in excels if "search-space" in p.name][0]
search_space = pd.read_excel(search_space).fillna("")
search_space
cols = ["model_name_or_path", "per_device_train_batch_size", "learning_rate", "num_train_epochs"]
print(search_space[cols].to_markdown())

|    | model_name_or_path    | per_device_train_batch_size   | learning_rate   | num_train_epochs   |
|---:|:----------------------|:------------------------------|:----------------|:-------------------|
|  0 | NbAiLab/nb-bert-base  | 32.0                          | 1e-05           | 12.0               |
|  1 | ltg/norbert3-large    | 64.0                          | 5e-05           |                    |
|  2 | NbAiLab/nb-bert-large |                               |                 |                    |


In [36]:
# Convert from latex to markdown
ltbl = """ & precision & recall & F1 & support \\
Neg & 0.444 & 0.190 & 0.267 & 21 \\
Neu & 0.674 & 0.955 & 0.790 & 132 \\
Pos & 0.882 & 0.479 & 0.621 & 94 \\
accuracy & 0.709 & 0.709 & 0.709 &  \\
macro avg & 0.667 & 0.541 & 0.559 & 247 \\
w. avg & 0.734 & 0.709 & 0.681 & 247 \\"""
ltxt = ltbl.replace("\\","").replace("&", ",").split("\n")
cols = None
rows = {}
for i, l in enumerate(ltxt):
    if i == 0:
        cols = l.split(",")[-4:]
    else:
        li = l.split(",")
        rows[li[0]] = li[1:]
rows
print(pd.DataFrame.from_dict(rows, orient="index", columns=cols).to_markdown())

|           |    precision  |    recall  |    F1  |  support    |
|:----------|--------------:|-----------:|-------:|:------------|
| Neg       |         0.444 |      0.19  |  0.267 | 21          |
| Neu       |         0.674 |      0.955 |  0.79  | 132         |
| Pos       |         0.882 |      0.479 |  0.621 | 94          |
| accuracy  |         0.709 |      0.709 |  0.709 |             |
| macro avg |         0.667 |      0.541 |  0.559 | 247         |
| w. avg    |         0.734 |      0.709 |  0.681 | 247         |
